In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install websockets
!pip install lxml

In [ ]:
import asyncio
import json
import uuid
import xml.etree.ElementTree as ET

import nest_asyncio
import pandas as pd
import websockets
from websockets import client

from local_secrets import ENTS, HOST, URI, secrets
from ws_client import generate_websocket_url
from tools import run_request_from_file
from order_config import CrossedTradesConfig, RemainingTradesConfig
from order_converter import OrderConverter
from xml_builder import BasketOrderXMLBuilder
from xml_parser import BasketOrderXMLParser
from order_types import SingleAllocation
from enums import (
    Side,
    OrderType,
    TimeInForce,
    SecurityIdType,
    CheckPretradeCompliance,
    FlowControlTag,
    ListProcessingLevel,
)

In [ ]:
nest_asyncio.apply()

In [ ]:
url = generate_websocket_url(URI, 'GET', secrets, HOST)

In [ ]:
os.environ['BBG_UUID'] = '26656679'
os.environ['PX_NUM'] = '4571'
os.environ['USE_NEW_CMGR_XML'] = 'TRUE'
os.environ['USING_IBM_MQ'] = 'FALSE'

In [ ]:
crossed_config = CrossedTradesConfig()
remaining_config = RemainingTradesConfig()
converter = OrderConverter(crossed_config, remaining_config)

crossed_path = "trades/crossed_trades_20251114_130344.csv"
remaining_path = "trades/remaining_trades_20251114_130344.csv"

crossed_df = pd.read_csv(crossed_path)
remaining_df = pd.read_csv(remaining_path)

# Combine both
# df_all = pd.concat([crossed_df, remaining_df], ignore_index=True, sort=False)

orders = converter.convert(crossed_df, remaining_df)

In [ ]:
orders[:3]

In [ ]:
crossed_orders = [o for o in orders if o['crossed']]
external_orders = [o for o in orders if not o['crossed']]
sample_orders = crossed_orders[:5] + external_orders[:5]
#sample_orders

In [ ]:
# Validate orders before submission
from order_submission_validator import OrderSubmissionValidator

print("\n" + "="*70)
print("VALIDATING ORDERS...")
print("="*70)

#is_valid, errors = OrderSubmissionValidator.validate_orders(orders)
is_valid, errors = OrderSubmissionValidator.validate_orders(orders)

if not is_valid:
    print("\nVALIDATION ERRORS FOUND:")
    for error in errors:
        print(f"  - {error}")
    print(f"\nTotal: {len(errors)} error(s)")
    print("\nPlease fix errors before submitting.")
    raise Exception("Order validation failed")
else:
    print("\nAll orders passed validation!")
    
    # Show summary
    summary = OrderSubmissionValidator.get_order_summary(orders)
    print(summary)

In [ ]:
# Generate XML
print("\nBuilding XML...")
xml_out = BasketOrderXMLBuilder.get_request_xml_string(
    custom_list_id=uuid.uuid1(),
    # list_of_orders=orders,
    list_of_orders=sample_orders,
    basket_name=None,
    basket_name_prefix="SETTLE_LE_BQuant_Demo",
    route_to_session="4571.DRAY.BQNT",
    check_pretrade_compliance=CheckPretradeCompliance.NO,
    flow_control_flag=FlowControlTag.ACTIVE_ORDER,
    list_processing_level=ListProcessingLevel.LIST,
    compliance_override_text="TestOverride",
)

# Save XML to a file
with open("xml_requests/basket_order_request.txt", "w", encoding="utf-8") as f:
    f.write(xml_out)
    
print("XML saved to basket_order_request.xml")
#print(xml_out)

In [ ]:
# Before sending, save and inspect the XML
with open("xml_requests/basket_order_request.txt", "r") as f:
    xml_content = f.read()

print("="*70)
print("XML REQUEST VALIDATION")
print("="*70)

# Verify XML is valid
try:
    root = ET.fromstring(xml_content)
    print("XML is well-formed")
    
    # Check critical fields
    basket_name = root.findtext(".//BasketName")
    tot_orders = root.findtext(".//TotNoOrders")
    pricing_no = root.findtext(".//PricingNo")
    uuid_val = root.findtext(".//UUID")
    
    print(f"BasketName: {basket_name}")
    print(f"TotNoOrders: {tot_orders}")
    print(f"PricingNo: {pricing_no}")
    print(f"UUID: {uuid_val}")
    
    # Count actual orders
    orders = root.findall(".//NoOrders/Order")
    print(f"Actual orders in XML: {len(orders)}")
    
    if int(tot_orders) != len(orders):
        print(f"WARNING: TotNoOrders ({tot_orders}) doesn't match actual orders ({len(orders)})")
    
except ET.ParseError as e:
    print(f"XML PARSE ERROR: {e}")
    print("Cannot send invalid XML to Bloomberg!")

In [ ]:
print("\n" + "="*70)
print("SENDING ORDER TO BLOOMBERG...")
print("="*70)

api_response_xml = run_request_from_file('xml_requests/basket_order_request.txt', url)

# DEBUG: Check what we got back
print(f"\nDEBUG: Type of api_response_xml: {type(api_response_xml)}")
print(f"DEBUG: Is None? {api_response_xml is None}")
print(f"DEBUG: Bool value? {bool(api_response_xml)}")

if api_response_xml is not None:
    print(f"DEBUG: Length: {len(api_response_xml)}")
    print(f"DEBUG: First 200 chars: {api_response_xml[:200]}")

In [ ]:
from order_status_checker import OrderStatusChecker
from ws_client import generate_websocket_url
from local_secrets import secrets, HOST, URI

# After sending orders (when api_response_xml is None)
if api_response_xml is None:
    print("\n" + "="*70)
    print("ORDER SENT - CHECKING STATUS")
    print("="*70)
    
    # Extract basket name from XML
    import xml.etree.ElementTree as ET
    with open('xml_requests/basket_order_request.txt', 'r') as f:
        xml_content = f.read()
    root = ET.fromstring(xml_content)
    basket_name = root.findtext(".//BasketName")
    
    print(f"Basket Name: {basket_name}")
    print(f"Querying Bloomberg for order status...\n")
    
    # GENERATE FRESH URL WITH NEW JWT TOKEN
    print("Generating fresh authentication token...")
    status_url = generate_websocket_url(URI, 'GET', secrets, HOST)
    
    # Create status checker
    checker = OrderStatusChecker()
    
    # Query for status with fresh URL
    status_response = checker.check_basket_status(
        url=status_url,  # Use fresh URL
        basket_name=basket_name,
        pricing_no="4571",
        uuid_val="26656679",
        max_retries=5,
        retry_delay=2.0
    )
    
    if status_response:
        print("\n" + "="*70)
        print("STATUS RESPONSE RECEIVED")
        print("="*70)
        
        # Parse the status response
        status_info = checker.parse_status_response(status_response)
        
        if status_info["success"]:
            print(f"List ID: {status_info['list_id']}")
            print(f"List Status: {status_info['list_status']}")
            print(f"Orders Found: {status_info['order_count']}")
            
            if status_info['orders']:
                print("\nIndividual Orders:")
                for idx, order in enumerate(status_info['orders'], 1):
                    print(f"  {idx}. {order['clordid']}")
                    print(f"     Order ID: {order['order_id']}")
                    print(f"     Status: {order['status']}")
                    print(f"     Security: {order['security']}")
            
            # Now parse with your regular parser
            print("\n" + "="*70)
            print("PARSING WITH STANDARD PARSER")
            print("="*70)
            
            from xml_parser import BasketOrderXMLParser
            from enums import ListProcessingLevel
            
            response = BasketOrderXMLParser.get_response_from_xml(
                xml_string=status_response,
                list_processing_level=ListProcessingLevel.LIST
            )
            
            print(f"Order ID: {response.order_id}")
            print(f"Status: {response.order_status.value} ({response.order_status.name})")
            print(f"Compliance: {response.compliance_response}")
            
        else:
            print(f"Error parsing response: {status_info['error']}")
            print(f"Raw XML (first 500 chars): {status_response[:500]}")
    else:
        print("\n" + "="*70)
        print("NO STATUS RESPONSE RECEIVED")
        print("="*70)
        print("Please manually verify orders in Bloomberg AIM.")
        print(f"Basket Name: {basket_name}")

In [ ]:
#run_request_from_file('Request-What-IF.txt', url)
#run_request_from_file('Request-Single.txt', url)
# run_request_from_file('test.txt', url)
#run_request_from_file('basket_order_request.txt', url)

In [ ]:
list_of_orders = [
    dict(
        security_id="NVDA US Equity",
        security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
        side=Side.BUY,
        order_type=OrderType.MARKET,
        # order_type=OrderType.LIMIT,
        # limit_price=145.12,
        time_in_force=TimeInForce.GOOD_TILL_CANCEL,
        quantity=1000,
        settl_currency="USD",
        allocation_instruction=[
            SingleAllocation(Account="TEST", Quantity=1000)
        ],
        crossed=True,   # <-- include broker (CROSS)
        instructions="This is a test note for crosses"
    ),
    dict(
        security_id="ABBV US Equity",
        security_id_type=SecurityIdType.BLOOMBERG_SYMBOL,
        side=Side.BUY,
        order_type=OrderType.LIMIT,
        limit_price=192.55,
        quantity=500,
        settl_currency="USD",
        allocation_instruction=[SingleAllocation(Account="TEST", Quantity=500)],
        crossed=False,  # <-- no broker / standard Parties block
    ),
]

xml_out = BasketOrderXMLBuilder.get_request_xml_string(
    custom_list_id=uuid.uuid1(),
    list_of_orders=list_of_orders,
    basket_name=None,
    basket_name_prefix="BQuantDemo",
    route_to_session="4571.DRAY.BQNT",
    check_pretrade_compliance=CheckPretradeCompliance.NO,
    flow_control_flag=FlowControlTag.ACTIVE_ORDER,
    list_processing_level=ListProcessingLevel.LIST,
    compliance_override_text="TestOverride",
)

with open("xml_requests/test.txt", "w", encoding="utf-8") as f:
    f.write(xml_out)

# print("XML Request built and saved to test.txt")
# print(xml_out)

In [ ]:
# Assuming we have 'url' variable set up from ws_client.generate_websocket_url()
# url = generate_websocket_url(...)

print("\n" + "="*70)
print("SENDING ORDER TO BLOOMBERG...")
print("="*70)

api_response_xml = run_request_from_file('xml_requests/test.txt', url)

In [ ]:
# parse response
if api_response_xml:
    print("\n" + "="*70)
    print("PARSING BLOOMBERG RESPONSE...")
    print("="*70)
    
    try:
        response = BasketOrderXMLParser.get_response_from_xml(
            xml_string=api_response_xml,
            list_processing_level=ListProcessingLevel.LIST
        )
        
        # display parsed results
        print("\n" + "="*70)
        print("ORDER SUBMISSION RESULTS")
        print("="*70)
        print(f"Order ID: {response.order_id}")
        print(f"Status: {response.order_status.value} ({response.order_status.name})")
        print(f"Compliance: {response.compliance_response}")
        
        if response.error_message:
            print(f"\nError: {response.error_message}")
        else:
            print(f"\nOrder submitted successfully!")
        
        # Display individual order responses
        if response.individual_responses:
            print(f"\nIndividual Order Details:")
            for idx, order_resp in enumerate(response.individual_responses, 1):
                print(f"  Order {idx}: Status = {order_resp.get('order_status', 'Unknown')}")
        
        print("="*70)
        
        # access reponse as dict
        response_dict = response.to_dict()
        print("\nResponse as dictionary:")
        print(response_dict)
        
    except Exception as e:
        print(f"\nFailed to parse response: {e}")
        print(f"Raw response: {api_response_xml[:500]}...")
else:
    print("\nNo response received from Bloomberg!")

In [ ]:
api_response_xml

In [ ]:
from order_validator import ValidationError

def example_validation_errors():
    """Example showing validation errors"""
    
    # Create DataFrame with intentional errors
    bad_df = pd.DataFrame({
        'security': ['AAPL US Equity', 'INVALID', 'TSLA US Equity'],
        'remaining_quantity': [100, -500, 0],  # 0 is invalid
        'portfolio_id': ['ACCT1', 'ACCT2', ''],  # Empty is invalid
        'trade_direction': ['BUY', 'SELL', 'BUY']
    })
    
    converter = OrderConverter(
        CrossedTradesConfig(),
        RemainingTradesConfig()
    )
    
    try:
        orders = converter.convert(None, bad_df)
    except ValidationError as e:
        print("Validation caught errors:")
        print(e)

In [ ]:
example_validation_errors()

In [ ]:
def example_with_custom_columns():
    """Example with custom column names"""
    
    # Your CSV has different column names
    crossed_df = pd.read_csv("my_crossed_trades.csv")
    
    # Configure column mapping
    crossed_config = CrossedTradesConfig(
        security_column="ticker",
        quantity_column="qty",
        buyer_column="buyer_acct",
        seller_column="seller_acct",
        cross_id_column="id"
    )
    
    remaining_config = RemainingTradesConfig(
        security_column="ticker",
        quantity_column="qty_remaining",
        portfolio_column="acct_id",
        side_column="direction"
    )
    
    converter = OrderConverter(crossed_config, remaining_config)
    orders = converter.convert(crossed_df, None)  # Only crossed trades
    
    print(f"Converted {len(orders)} orders")

In [ ]:
# example_with_custom_columns()

In [ ]:
# Test 1: Market order (crossed)
crossed_order = {
    'security_id': 'AAPL US Equity',
    'security_id_type': SecurityIdType.BLOOMBERG_SYMBOL,
    'side': Side.BUY,
    'order_type': OrderType.MARKET,  # No price needed
    'quantity': 100,
    'settl_currency': 'USD',
    'allocation_instruction': [SingleAllocation(Account='TEST', Quantity=100)],
    'crossed': True,
    'instructions': 'CROSS_ID:12345 | BUYER:P-001 | SELLER:P-002'
}

# Test 2: Limit order
limit_order = {
    'security_id': 'TSLA US Equity',
    'security_id_type': SecurityIdType.BLOOMBERG_SYMBOL,
    'side': Side.BUY,
    'order_type': OrderType.LIMIT,
    'limit_price': 250.00,  # Price required
    'quantity': 50,
    'settl_currency': 'USD',
    'allocation_instruction': [SingleAllocation(Account='TEST', Quantity=50)],
    'crossed': False,
}

# Test 3: Stop-limit order
stop_limit_order = {
    'security_id': 'NVDA US Equity',
    'security_id_type': SecurityIdType.BLOOMBERG_SYMBOL,
    'side': Side.SELL,
    'order_type': OrderType.STOP_LIMIT,
    'limit_price': 140.00,
    'stop_price': 145.00,
    'quantity': 75,
    'settl_currency': 'USD',
    'allocation_instruction': [SingleAllocation(Account='TEST', Quantity=75)],
    'crossed': False,
}

In [ ]:
xml_out = BasketOrderXMLBuilder.get_request_xml_string(
    custom_list_id=uuid.uuid1(),
    list_of_orders=[crossed_order, limit_order, stop_limit_order],
    basket_name=None,
    basket_name_prefix="LE_BQuantDemo",
    route_to_session="4571.DRAY.BQNT",
    check_pretrade_compliance=CheckPretradeCompliance.NO,
    flow_control_flag=FlowControlTag.ACTIVE_ORDER,
    list_processing_level=ListProcessingLevel.LIST,
    compliance_override_text="TestOverride",
)

with open("xml_requests/test.txt", "w", encoding="utf-8") as f:
    f.write(xml_out)